In [1]:
'''
@author : SOHAM PATHAK

'''
import pytesseract
pytesseract.pytesseract.tesseract_cmd ='D:/Tesseract-OCR/tesseract'
%pylab inline
import cv2
import os
directory = 'C:/Users/KIIT_Intern/Desktop/ML-DS/fEATURE Engg/costa-rican-household-poverty-prediction'


#<----------------------------------------------->
last_check = '8/11/2019'

Populating the interactive namespace from numpy and matplotlib


In [2]:
def helper_function(res):
    txt =pytesseract.image_to_string(res)
    
    #indices = [i for i, x in enumerate(cd) if x == "xx"]
    ##preprocessing
    
    result1 = re.sub(r"\n", " ", txt, flags=re.I) 
    result2 = re.sub(r"\\", "", result1)
    result3 = re.sub(r"|", "", result2)
    txt = re.sub(r"¢", "0", result3)
    
    return txt
##<---------------------------------------------------------------------------------------------------------->##
def saviour_function(res,lst):
    txt =pytesseract.image_to_string(res)
    
    indices = [i for i, x in enumerate(lst) if x == "xx"]
    
    
    
    ##preprocessing
    
    result1 = re.sub(r"\n", " ", txt, flags=re.I) 
    result2 = re.sub(r"\\", "", result1)
    result3 = re.sub(r"|", "", result2)
    txt = re.sub(r"¢", "0", result3) 
    ##date
    try:
        if 0 in set(indices):
            lst[0] = 'Date: '+re.search(r'(\d+(/|-){1}\d+(/|-){1}\d{2,4})', txt)[0]
    except:
        pass   
    #time
    try:
        if 0 in set(indices):
            lst[1] = "Time:"+re.search(r'(\d+:\d+:\d+)', txt)[0]
    except:
        pass
    ##scan completed or not
    try:
        if (2 in set(indices)):
            if re.findall('Completed', txt):
                lst[2] = 'completed'
            else:
                lst[2]='error'
    except:
        pass
    ##risk
    try:
        if 3 in set(indices):
            lst[3] = re.search(r"Risks found:(?:[^a-zA-Z'-]+){1}", txt)[0]
        
    except:
        pass
    ##files scanned
    try:
        if 4 in set(indices):
            lst[4] = re.search(r"scanned:(?:[^a-zA-Z'-]+){1}", txt)[0]
    except:
        pass 
    ##hostname
    try:
        if 5 in set(indices):
            lst[5] = re.search(r"KIIT_(?:[^a-zA-Z'-]){5}", txt)[0][5:]
    except:
        pass
    
    return lst

##<------------------------------------------------------------------------------------------------------------->##

def data_retriver(img):
    lst = ['xx','xx','xx','xx','xx','xx']
    ##<--------------------2XSCALING--------------------->
    scale_percent = 200 # percent of original size
    width = int(img.shape[1] * scale_percent / 100)
    height = int(img.shape[0] * scale_percent / 100)
    dim = (width, height)
    res = cv2.resize(img, dim, interpolation = cv2.INTER_AREA)

    ##<----------------------------------------------------->
    ret,thresh2 = cv2.threshold(res,127,255,cv2.THRESH_BINARY_INV)
    txt = helper_function(thresh2)
    try:
        lst[5] = re.search(r"KIIT_(?:[^a-zA-Z'-]){5}", txt)[0][5:]
    except:
        pass 
    
    ##<-------------------------------------------------------->
    rest = cv2.cvtColor(res, cv2.COLOR_BGR2GRAY)
    th1 = cv2.adaptiveThreshold(rest,255,cv2.ADAPTIVE_THRESH_MEAN_C,cv2.THRESH_BINARY,11,2)
    txt2 = helper_function(th1)
    ##date
    try:
        
        lst[0] = 'Date: '+re.search(r'(\d+(/|-){1}\d+(/|-){1}\d{2,4})', txt2)[0]
    except:
        pass
     #time
    try:
        
        lst[1] = "Time:"+re.search(r'(\d+:\d+:\d+)', txt2)[0]
    except:
        pass  
    
   
    ##<------------------------------------------------------------>
    ##LAB
    lab= cv2.cvtColor(res, cv2.COLOR_BGR2LAB)
    l, a, b = cv2.split(lab)
    clahe = cv2.createCLAHE(clipLimit=10.0, tileGridSize=(10,10))
    cl = clahe.apply(l)
    limg = cv2.merge((cl,a,b))
    txt3 = helper_function(limg)
    ##scan completed or not
    try:
        if re.findall('Completed', txt3):
            lst[2] = 'completed'
        else:
            lst[2]='error'
    except:
        pass
    
    ##risk
    try:
        lst[3] = re.search(r"Risks found:(?:[^a-zA-Z'-]+){1}", txt3)[0]
        
    except:
        pass

    ##files scanned
    try:
        lst[4] = re.search(r"scanned:(?:[^a-zA-Z'-]+){1}", txt3)[0]
    except:
        pass   
    
    indices = [i for i, x in enumerate(lst) if x == "xx"]
    if len(indices)>3:
        print('Unable to detect properly !! Please rescan')
        
    if len(indices)==1:
        th1 = cv2.adaptiveThreshold(rest,255,cv2.ADAPTIVE_THRESH_MEAN_C,cv2.THRESH_BINARY,11,2)
        lst = saviour_function(th1,lst)

    return lst

##<--------------------------------------------------------------------------------------------------------------->

def isvalid_fun(lstr):
    try:
        scan_date = datetime.datetime.strptime(lstr[0][6:], '%m/%d/%Y').strftime('%m/%d/%Y')
    except:
        scan_date = datetime.datetime.strptime(lstr[0][6:], '%d-%m-%Y').strftime('%d/%m/%Y')
        
    last_security_check = datetime.datetime.strptime(last_check, '%m/%d/%Y').strftime('%m/%d/%Y')
    if((scan_date > last_security_check)&(lstr[2]=='completed')):
        return '1'
    else:
        return '0'

In [3]:
%%time
filename = "data_1.csv"
f = open(filename,"w")
headers = "Empid ,Files_Scanned,Risk_Files,Completed,Date,Time,filename,isValid \n"
f.write(headers)

for filename in os.listdir(directory):
    if filename.endswith(".png"):
        img_0 = cv2.imread(filename)
        lstr = data_retriver(img_0)
        isvalid = isvalid_fun(lstr)
        try :
            #writing onto the file
            f.write(lstr[5] + "," + lstr[4][8:].replace(",","") + "," + lstr[3][13:]+ "," + lstr[2]+ "," + lstr[1][5:]+ "," + lstr[0][5:]+ "," + filename+ "," + isvalid + "\n")
        #handle no attribute exception which will occur if the specific tag is not present int he tag
        except IndexError as e:
            print("nil") 
f.close()             

Wall time: 3min 27s
